In [ ]:
!pip install --quiet --upgrade pip
!pip install --quiet transformers torch scipy
!pip install --quiet faster-whisper
!pip install --quiet flask flask-cors
!pip install --quiet pyngrok
!pip install --quiet torch
!pip install --quiet numpy
!pip install --quiet requests
!pip install --quiet accelerate
!pip install --quiet optimum

from flask import Flask, request, jsonify
import base64
from io import BytesIO
from pyngrok import ngrok
from flask_cors import CORS
import torch
import scipy.io.wavfile
import requests
import warnings
from faster_whisper import WhisperModel
import os
import numpy as np
import json
import logging
from typing import List, Dict
import transformers
warnings.filterwarnings('ignore')
transformers.logging.set_verbosity_error()

import uuid
import time
import sys
import json
from pprint import pformat

# Configure logging for Angelo's reel generation
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[
        logging.FileHandler("whisper_reel_transcription.log"),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

print("🎤 Initializing Whisper service for Angelo's Reel Generator...")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Initialize Whisper model for Italian transcription
model = WhisperModel(
    model_size_or_path="base",
    device=device,
    compute_type="float16" if device == "cuda" else "int8"
)
print("✅ Whisper model loaded successfully")

def process_audio_with_whisper(wav_file_path, request_id=None):
    """
    Whisper implementation optimized for Italian reel generation
    Returns word-level and line-level timestamps for subtitle creation
    """
    if request_id is None:
        request_id = str(uuid.uuid4())[:8]

    start_time = time.time()
    print(f"[WHISPER-{request_id}] Starting Italian audio transcription")

    try:
        file_size = os.path.getsize(wav_file_path)
        print(f"[WHISPER-{request_id}] Processing audio file: {wav_file_path} ({file_size/1024:.2f} KB)")
    except Exception as e:
        print(f"[WHISPER-{request_id}] Error getting file info: {str(e)}")

    print(f"[WHISPER-{request_id}] Using model: base on {device}")

    try:
        if torch.cuda.is_available():
            memory_before = torch.cuda.memory_allocated()
            print(f"[WHISPER-{request_id}] Initial GPU memory: {memory_before/1024**2:.2f} MB")

        # Transcribe with Italian language preference
        print(f"[WHISPER-{request_id}] Starting transcription with Italian language...")
        transcription_start = time.time()
        segments, info = model.transcribe(
            wav_file_path,
            beam_size=5,
            word_timestamps=True,
            language="it"  # Force Italian for Angelo's content
        )

        segments = list(segments)
        transcription_time = time.time() - transcription_start

        print(f"[WHISPER-{request_id}] ✅ Transcription completed in {transcription_time:.2f}s")
        print(f"[WHISPER-{request_id}] Detected language: {info.language} (probability: {info.language_probability:.4f})")
        print(f"[WHISPER-{request_id}] Number of segments: {len(segments)}")

        # Process word-level results for subtitle timing
        print(f"[WHISPER-{request_id}] Processing word-level timestamps for subtitles...")
        word_level_results = []
        total_words = 0

        for segment_idx, segment in enumerate(segments):
            segment_words = list(segment.words)
            total_words += len(segment_words)

            print(f"[WHISPER-{request_id}] Segment {segment_idx+1}: {len(segment_words)} words, {segment.start:.2f}s to {segment.end:.2f}s")
            print(f"[WHISPER-{request_id}] Text: \"{segment.text}\"")

            for word in segment_words:
                word_level_results.append({
                    "word": word.word.strip(),
                    "start": round(word.start, 2),
                    "end": round(word.end, 2)
                })

        print(f"[WHISPER-{request_id}] Total words detected: {total_words}")

        # Create line-level grouping optimized for TikTok/Reels (shorter lines)
        print(f"[WHISPER-{request_id}] Creating subtitle lines for mobile viewing...")
        line_level_results = []
        current_line = []
        current_text = []

        # Optimized for mobile/vertical video viewing
        MAX_CHARS = 25  # Shorter for mobile screens
        MAX_DURATION = 2.5  # Good for TikTok pacing
        MAX_GAP = 1.5

        print(f"[WHISPER-{request_id}] Line parameters: MAX_CHARS={MAX_CHARS}, MAX_DURATION={MAX_DURATION}s, MAX_GAP={MAX_GAP}s")

        for idx, word in enumerate(word_level_results):
            current_line.append(word)
            current_text.append(word["word"])

            line_text = " ".join(current_text)
            line_duration = word["end"] - current_line[0]["start"]

            should_break = False

            if len(line_text) > MAX_CHARS:
                should_break = True
            elif line_duration > MAX_DURATION:
                should_break = True
            elif idx > 0:
                gap = word["start"] - word_level_results[idx-1]["end"]
                if gap > MAX_GAP:
                    should_break = True

            if should_break or idx == len(word_level_results) - 1:
                if current_line:
                    new_line = {
                        "text": " ".join(current_text),
                        "start": current_line[0]["start"],
                        "end": current_line[-1]["end"],
                        "words": current_line.copy()
                    }
                    line_level_results.append(new_line)
                    current_line = []
                    current_text = []

        print(f"[WHISPER-{request_id}] Created {len(line_level_results)} subtitle lines")

        # Show sample lines for verification
        if line_level_results:
            sample_lines = min(3, len(line_level_results))
            print(f"[WHISPER-{request_id}] Sample subtitle lines:")
            for i in range(sample_lines):
                print(f"  Line {i+1}: \"{line_level_results[i]['text']}\"")

        # Prepare results for reel generation
        results = {
            "word_level": word_level_results,
            "line_level": line_level_results,
            "detected_language": info.language,
            "language_probability": info.language_probability,
            "processing_time": {
                "transcription": transcription_time,
                "total": time.time() - start_time
            }
        }

        if torch.cuda.is_available():
            memory_after = torch.cuda.memory_allocated()
            print(f"[WHISPER-{request_id}] GPU memory after: {memory_after/1024**2:.2f} MB")

        print(f"[WHISPER-{request_id}] ✓ Ready for reel generation!")
        return results

    except Exception as e:
        print(f"[WHISPER-{request_id}] ❌ ERROR: {str(e)}")
        raise

    finally:
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        print("-" * 50)

# Flask app for ngrok exposure
app = Flask(__name__)
CORS(app)

@app.route('/process_audio', methods=["POST"])
def process_audio():
    """Process audio for Angelo's reel generation pipeline"""
    request_id = str(uuid.uuid4())[:8]
    start_time = time.time()

    print(f"[REEL-WHISPER-{request_id}] Received audio processing request")

    data = request.json
    if 'audio_data' not in data:
        print(f"[REEL-WHISPER-{request_id}] ERROR: No audio data in request")
        return jsonify({"error": "No audio data provided"}), 400

    audio_base64 = data['audio_data']
    print(f"[REEL-WHISPER-{request_id}] Audio data size: {len(audio_base64)/1024:.2f} KB")

    temp_path = f"temp_reel_audio_{request_id}.wav"
    print(f"[REEL-WHISPER-{request_id}] Saving to: {temp_path}")

    try:
        # Decode and save audio
        audio_bytes = base64.b64decode(audio_base64)
        with open(temp_path, "wb") as f:
            f.write(audio_bytes)

        file_size = os.path.getsize(temp_path)
        print(f"[REEL-WHISPER-{request_id}] File created: {file_size/1024:.2f} KB")

        # Process with Whisper
        results = process_audio_with_whisper(temp_path, request_id)

        # Summary for reel generation
        word_count = len(results["word_level"])
        line_count = len(results["line_level"])
        
        print(f"[REEL-WHISPER-{request_id}] Results for reel generation:")
        print(f"  - Words: {word_count}")
        print(f"  - Subtitle lines: {line_count}")
        print(f"  - Language: {results['detected_language']}")
        
        if word_count > 0:
            audio_duration = results["word_level"][-1]["end"]
            print(f"  - Duration: {audio_duration:.2f}s")

        total_time = time.time() - start_time
        print(f"[REEL-WHISPER-{request_id}] ✓ Processing complete in {total_time:.2f}s")

        return jsonify(results)

    except Exception as e:
        print(f"[REEL-WHISPER-{request_id}] ❌ ERROR: {str(e)}")
        return jsonify({"error": str(e)}), 500

    finally:
        if os.path.exists(temp_path):
            os.remove(temp_path)
            print(f"[REEL-WHISPER-{request_id}] Temp file cleaned up")

# Ngrok setup for external access
def create_app():
    """Create ngrok tunnel for Angelo's reel generation service"""
    try:
        print("🌐 Setting up ngrok tunnel for Whisper service...")
        ngrok.set_auth_token("")
        public_url = ngrok.connect(5000).public_url
        
        print(f"✅ Whisper service available at: {public_url}")
        print(f"   Endpoint: {public_url}/process_audio")
        print(f"   Use this URL in your WHISPER_NGROK_URL environment variable")
        
        # Optional: notify Django service (if running)
        try:
            response = requests.post(
                'https://c8e168cf22ff.ngrok-free.app/update-ngrok-url3/',
                json={"ngrok_url": public_url}
            )
            logger.info("Ngrok URL sent to Django service")
        except Exception as e:
            logger.info(f"Django notification failed (this is optional): {e}")

        return app, public_url

    except Exception as e:
        print(f"❌ Ngrok setup failed: {e}")
        return None, None

# Start the service
print("🚀 Starting Whisper service for Angelo's Reel Generator...")
if __name__ == '__main__':
    app, url = create_app()
    if app:
        print("🎤 Whisper service is ready for reel generation!")
        app.run(port=5000)
    else:
        print("❌ Failed to start Whisper service")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.7 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
libcugraph-cu12 25.6.0 requires libraft-cu12==25.6.*, but you have libraft-cu12 25.2.0 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.
pylibcugraph-cu12 25.6.0 requires pylibraft-cu12==25.6.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 25.6.0 requires rmm-cu12==25.6.*, but you have rmm-cu12 25.2.0 which is incompatible.
🎤 Initializing Whisper service for Angelo's Reel Generator...
Using device: cuda


vocabulary.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/145M [00:00<?, ?B/s]

✅ Whisper model loaded successfully
🚀 Starting Whisper service for Angelo's Reel Generator...
🌐 Setting up ngrok tunnel for Whisper service...
✅ Whisper service available at: https://60c266720c54.ngrok-free.app                                 
   Endpoint: https://60c266720c54.ngrok-free.app/process_audio
   Use this URL in your WHISPER_NGROK_URL environment variable
🎤 Whisper service is ready for reel generation!
 * Serving Flask app '__main__'
 * Debug mode: off
[REEL-WHISPER-fde774fe] Received audio processing request
[REEL-WHISPER-fde774fe] Audio data size: 930.67 KB
[REEL-WHISPER-fde774fe] Saving to: temp_reel_audio_fde774fe.wav
[REEL-WHISPER-fde774fe] File created: 698.00 KB
[WHISPER-fde774fe] Starting Italian audio transcription
[WHISPER-fde774fe] Processing audio file: temp_reel_audio_fde774fe.wav (698.00 KB)
[WHISPER-fde774fe] Using model: base on cuda
[WHISPER-fde774fe] Initial GPU memory: 0.00 MB
[WHISPER-fde774fe] Starting transcription with Italian language...
[WHISPER-fde